In [3]:
with open(f"./small_dgemv_naive_c_implementation_ver2-0_YmmNaive.c", "w") as p:

    p.write("#include <stddef.h>\n")
    p.write("#include <stdio.h>\n")
    p.write("#include <stdlib.h>\n")
    p.write("#include <stdint.h>\n")
    p.write("#include <math.h>\n")
    p.write("#include <omp.h>\n")
    p.write("#include <immintrin.h>\n") 
    p.write("#include <xmmintrin.h>\n")     

    code = f"""        
double sum_ymm_elements(__m256d ymmX){{
    __m128d xmm_upper = _mm256_extractf128_pd(ymmX, 1);
    __m128d xmm_lower = _mm256_extractf128_pd(ymmX, 0);

    __m128d xmm_sum = _mm_add_pd(xmm_upper, xmm_lower);

    __m128d xmm_high_low = _mm_add_pd(xmm_sum, _mm_unpackhi_pd(xmm_sum, xmm_sum));

    double result;
    _mm_store_sd(&result, xmm_high_low);

    return result;
}}    
    
    

void mydgemv_n_ver2_0(double a[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    __m256d ymm0, ymm1, ymm2;
    double tmp_x;
    for (int64_t i=0; i<lda; i++){{
        y[i] = 0.0;
    }}
    
    int64_t i_remain = lda % 4;
    for (int64_t k=0; k<ldx; k++){{
        ymm0 = _mm256_set1_pd(x[k]);
        for (int64_t i=0; i<lda-i_remain; i+=4){{
            ymm1 = _mm256_loadu_pd(&y[i]);
            ymm2 = _mm256_loadu_pd(&a[i]);
            
            ymm2 = _mm256_mul_pd(ymm0, ymm2);
            
            ymm1 = _mm256_add_pd(ymm1, ymm2);
            
            _mm256_storeu_pd(&y[i], ymm1);
        }}
        
        double tmp_x = x[k];
        for (int64_t i=lda-i_remain; i<lda; i++){{
            y[i] += a[i] * tmp_x;
        }}
        a += lda;
    }}
}}


void mydgemv_t_ver2_0(double a_t[], double x[], double y[], int64_t lda, int64_t ldx, int64_t ldy){{
    __m256d ymm0, ymm1, ymm2;
    
    int64_t k_remain = ldx % 4;
    for (int64_t i=0; i<lda; i++){{
        ymm0 = _mm256_setzero_pd();
        
        for (int64_t k=0; k<ldx-k_remain; k+=4){{
            ymm1 = _mm256_loadu_pd(&x[k]);
            ymm2 = _mm256_loadu_pd(&a_t[k]);
            
            ymm1 = _mm256_mul_pd(ymm1, ymm2);
            
            ymm0 = _mm256_add_pd(ymm0, ymm1);
        }}
        
        y[i] = sum_ymm_elements(ymm0);
        a_t += ldx;
    }}
}}
    """
    p.write(code)